<a href="https://colab.research.google.com/github/xjdeng/mbtimodel/blob/main/mbti_model_reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/xjdeng/mbtimodel
!pip install praw
!pip install -r mbtimodel/requirements.txt

Cloning into 'mbtimodel'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 24 (delta 6), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (24/24), 25.19 MiB | 13.14 MiB/s, done.
Resolving deltas: 100% (6/6), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.7 MB/s eta 0:00:00


In [2]:
import joblib
import praw
from google.colab import userdata
import bs4
import markdown
import re
import pandas as pd
import praw
from prawcore.exceptions import NotFound
import string
table = str.maketrans('', '', string.punctuation)

In [3]:
model = joblib.load("mbtimodel/mbtimodel.pkl")
client_id = userdata.get("reddit_client_id")
assert client_id is not None
client_secret = userdata.get("reddit_client_secret")
assert client_secret is not None
username = userdata.get("reddit_username")
assert username is not None
password = userdata.get("reddit_password")
assert password is not None
app_name = userdata.get("reddit_app")
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=app_name,
                     username=username, \
                     password=password)

with open("mbtimodel/english10000.txt",'r') as f:
    words = f.read().split("\n")
wordset = set(words)
indexdict = {None: 0}
for i,w in enumerate(words):
    indexdict[w] = i
def get_distribution(txt):
    txtwords = txt.translate(table).lower().split()
    dist = [0]*(len(words))
    for w in txtwords:
        if w in wordset:
            dist[indexdict[w]] += 1
    tot = sum(dist)
    if tot == 0:
        return [0]*(len(words))
    return [d/tot for d in dist]

In [4]:
def noquotes(text):
    """
This function first stated out as a way to remove markdown quotes from raw reddit markdown text but now it's more of a
general purpose text parser, but the name hasn't changed.
    """
    #https://stackoverflow.com/questions/761824/python-how-to-convert-markdown-formatted-text-to-text
    html = markdown.markdown(text)
    text = ''.join(bs4.BeautifulSoup(html, 'lxml').findAll(string=True))
    t1 = re.sub(">.+?(\n|$)","",text).replace("\\n","").replace("\\","")
    return t1

def predict_mbti_txt(txt):
    dists = get_distribution(txt)
    proba = model.predict_proba([dists])[0]
    classes = model.classes_
    output = pd.DataFrame()
    output['Personality'] = classes
    output['Probability'] = proba
    output.sort_values('Probability', inplace=True, ascending = False)
    return output

def predict_user(username):
    try:
        comms = list(reddit.redditor(username).comments.new(limit=None))
        subs = list(reddit.redditor(username).submissions.new(limit=None))
        text = []
        ups = 0
        downs = 0
        for comment in comms:
            text.append(noquotes(comment.body))
            votes = comment.ups - 1
            if comment.controversiality == 1:
                extent = abs(votes)*1.5 + 3
                ups += int(round(votes + extent))
                downs += int(round(votes - extent))
            else:
                if votes > 0:
                    ups += votes
                else:
                    downs += votes
        for sub in subs:
            newsub = noquotes(sub.selftext)
            if len(newsub) > 0:
                text.append(newsub)
        fulltext = " ".join(text)
        A = abs(ups)
        B = abs(downs)
        controversiality = B/(A+B)
        return predict_mbti_txt(fulltext), controversiality
    except NotFound:
        return None, None

In [6]:
predict_user("thisisbillgates")

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

(   Personality   Probability
 7         ESTP  1.000000e+00
 10        INTJ  4.020477e-26
 8         INFJ  3.591666e-26
 11        INTP  2.872673e-26
 9         INFP  2.343722e-26
 1         ENFP  1.472749e-26
 3         ENTP  1.372623e-26
 15        ISTP  6.764860e-27
 6         ESTJ  5.860599e-27
 13        ISFP  4.856838e-27
 2         ENTJ  3.813325e-27
 14        ISTJ  3.193561e-27
 0         ENFJ  2.668864e-27
 12        ISFJ  1.335680e-27
 4         ESFJ  5.138316e-29
 5         ESFP  4.202982e-30,
 0.0)